In [1]:
import numpy as np
import requests
import pytest
import tqdm
import ipytest

In [2]:
ipytest.autoconfig()

# 4) Доп. задание. Судоку

В данном задании нужно будет реализовать солвер судоку, поработать с API и написать тесты

Для начала научимся создавать матрицы 9x9 в numpy. Тебе в этом поможет модуль:
[numpy.random](https://numpy.org/doc/2.0/reference/random/generated/numpy.random.randint.html)

In [3]:
sudoku = np.random.randint(0, 9, size=(9, 9))
sudoku

array([[6, 3, 1, 7, 3, 4, 0, 6, 1],
       [8, 3, 0, 0, 3, 6, 1, 5, 7],
       [2, 2, 5, 7, 7, 2, 3, 5, 6],
       [0, 5, 1, 1, 5, 1, 6, 6, 2],
       [7, 4, 7, 6, 4, 3, 2, 6, 6],
       [3, 7, 0, 0, 0, 6, 2, 7, 1],
       [2, 1, 2, 6, 6, 2, 1, 5, 5],
       [7, 8, 5, 0, 5, 6, 1, 3, 1],
       [2, 2, 7, 3, 8, 8, 2, 6, 4]])

Теперь нужно проверить, что это матрица является [судоку](https://ru.wikipedia.org/wiki/%D0%A1%D1%83%D0%B4%D0%BE%D0%BA%D1%83)

## Валидация

In [4]:
def checker(matrix):
    # Считаем кол-во элементов в строке, строчке или квадрате
    values, counts = np.unique(matrix, return_counts=True) 
    for i in range(len(values)):
        if values[i] != 0 and counts[i] != 1:
            return False
    return True


def validate_sudoku(sudoku):
    # Проверка строк
    for row in range(9):
        if not(checker(sudoku[row])):
            return False
            
    # Проверка столбцов
    for col in range(9):
        if not(checker(sudoku[:, col])):
            return False
            
    # Проверка квадратов
    for row in range(0, 9, 3):
        for col in range(0, 9, 3):
            if not(checker(sudoku[row:row + 3, col: col + 3])):
                return False
    return True

In [5]:
# while not validate_sudoku(sudoku):
    # sudoku = np.random.randint(0, 9, size=(9, 9))
# sudoku

Понятно, что код выше выполняется очень долго. Напишем более оптимизированный генератор матриц судоку.

1) Сгенерируй матрицу, заполненную числами от 1 до 9, которая удовлетворяет validate_sudoku
2) Напиши функции:
  - Транспонирование матрицы
  - Обмен строк
  - Обмен столбцов
3) Примени эти функции в случайном порядке на сгенерированную матрицу и удали случайные клетки из нее
4) Проверь, является ли эта матрица судоку

## Генератор

In [6]:
def transpose(matrix):
    return np.transpose(matrix)

def swap_row(matrix, row1, row2):
    matrix[row1], matrix[row2] = matrix[row2].copy(), matrix[row1].copy()
    return matrix

def swap_col(matrix, col1, col2):
    matrix[:, col1], matrix[:, col2] = matrix[:, col2].copy(), matrix[:, col1].copy()
    return matrix

In [7]:
def generate_sudoku(n=9, r=10, d=50):
    # Гененрируем заполненную матрицу
    base = np.arange(1, n+1)
    sudoku = np.zeros((n, n), dtype=int)
    for i in range(n):
        sudoku[i] = np.roll(base, i)
    # Случайно тасуем строки, столбцы и транспонируем
    for i in range(r):
        s = np.random.randint(0, 2)
        if s == 0:
            sudoku = transpose(sudoku)
        elif s == 1:
            sudoku = swap_row(sudoku, np.random.randint(0, 9), np.random.randint(0, 9))
        elif s == 2:
            sudoku = swap_col(sudoku, np.random.randint(0, 9), np.random.randint(0, 9))
    # Удаляем некоторые числа
    for i in np.random.permutation(81)[:50]:
        sudoku[i % 9][i // 9] = 0
        
    return sudoku

sudoku = generate_sudoku()
sudoku

array([[0, 0, 3, 5, 9, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [4, 0, 6, 0, 0, 0, 0, 2, 0],
       [0, 0, 0, 0, 0, 0, 4, 5, 0],
       [6, 0, 8, 0, 5, 0, 0, 4, 9],
       [3, 0, 5, 0, 2, 8, 9, 0, 6],
       [8, 0, 0, 3, 0, 4, 0, 0, 2],
       [5, 0, 7, 9, 0, 0, 2, 0, 0],
       [9, 0, 2, 4, 8, 0, 0, 0, 0]])

In [8]:
while not validate_sudoku(sudoku):
    sudoku = generate_sudoku()
sudoku

array([[0, 2, 0, 0, 5, 0, 0, 0, 0],
       [5, 0, 3, 6, 0, 0, 0, 1, 0],
       [9, 8, 7, 1, 2, 3, 0, 0, 0],
       [1, 9, 0, 0, 0, 4, 0, 0, 0],
       [0, 0, 0, 0, 1, 2, 0, 0, 0],
       [7, 6, 0, 0, 9, 0, 0, 0, 0],
       [0, 0, 9, 0, 4, 0, 6, 0, 0],
       [0, 3, 2, 5, 0, 7, 8, 0, 0],
       [6, 0, 0, 0, 0, 0, 0, 2, 3]])

Теперь судоку генерируется быстрее

Теперь нужно будет написать солвер для судоку. Для того, чтобы проверить его работу ты можешь воспользоваться [API](https://www.youdosudoku.com/)

In [9]:
sudoku = requests.get('https://www.youdosudoku.com/api').json()
sudoku

{'difficulty': 'easy',
 'puzzle': '000002006798006000020489050306194080084603970010758300009060400000035700000200639',
 'solution': '145372896798516243623489157376194582584623971912758364239867415461935728857241639'}

In [10]:
not_puzzled = [int(i) for i in sudoku['puzzle']]
not_puzzled = np.array(not_puzzled).reshape((9, 9))
not_puzzled

array([[0, 0, 0, 0, 0, 2, 0, 0, 6],
       [7, 9, 8, 0, 0, 6, 0, 0, 0],
       [0, 2, 0, 4, 8, 9, 0, 5, 0],
       [3, 0, 6, 1, 9, 4, 0, 8, 0],
       [0, 8, 4, 6, 0, 3, 9, 7, 0],
       [0, 1, 0, 7, 5, 8, 3, 0, 0],
       [0, 0, 9, 0, 6, 0, 4, 0, 0],
       [0, 0, 0, 0, 3, 5, 7, 0, 0],
       [0, 0, 0, 2, 0, 0, 6, 3, 9]])

In [11]:
solution = [int(i) for i in sudoku['solution']]
solution = np.array(solution).reshape((9, 9))
solution

array([[1, 4, 5, 3, 7, 2, 8, 9, 6],
       [7, 9, 8, 5, 1, 6, 2, 4, 3],
       [6, 2, 3, 4, 8, 9, 1, 5, 7],
       [3, 7, 6, 1, 9, 4, 5, 8, 2],
       [5, 8, 4, 6, 2, 3, 9, 7, 1],
       [9, 1, 2, 7, 5, 8, 3, 6, 4],
       [2, 3, 9, 8, 6, 7, 4, 1, 5],
       [4, 6, 1, 9, 3, 5, 7, 2, 8],
       [8, 5, 7, 2, 4, 1, 6, 3, 9]])

In [12]:
validate_sudoku(not_puzzled)

True

## Солвер

In [13]:
def check_win(sudoku):
    if validate_sudoku(sudoku):
        values, counts = np.unique(sudoku, return_counts=True)
        if np.all(values != 0) and np.all(counts == 9):
            return True
    return False

In [14]:
def solver_sudoku(sudoku):
    for row in range(9):
        for col in range(9):
            if sudoku[row][col] == 0:
                for n in range(1, 10):
                    sudoku[row][col] = n
                    if validate_sudoku(sudoku):
                        if solver_sudoku(sudoku):
                            return True
                    sudoku[row][col] = 0
                return False
    return True

In [15]:
solve = not_puzzled.copy()
solver_sudoku(solve)
solve

array([[1, 4, 5, 3, 7, 2, 8, 9, 6],
       [7, 9, 8, 5, 1, 6, 2, 4, 3],
       [6, 2, 3, 4, 8, 9, 1, 5, 7],
       [3, 7, 6, 1, 9, 4, 5, 8, 2],
       [5, 8, 4, 6, 2, 3, 9, 7, 1],
       [9, 1, 2, 7, 5, 8, 3, 6, 4],
       [2, 3, 9, 8, 6, 7, 4, 1, 5],
       [4, 6, 1, 9, 3, 5, 7, 2, 8],
       [8, 5, 7, 2, 4, 1, 6, 3, 9]])

## Тесты

Теперь напиши тесты для солвера. Для сравнения матриц с ответом, можешь воспользоваться [assert](https://numpy.org/doc/2.0/reference/generated/numpy.testing.assert_array_equal.html#numpy.testing.assert_array_equal)

In [16]:
%%ipytest -qq


def test_solve_sudoku():
    input_board = np.array([
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ])
    
    expected_solution = np.array([
        [5, 3, 4, 6, 7, 8, 9, 1, 2],
        [6, 7, 2, 1, 9, 5, 3, 4, 8],
        [1, 9, 8, 3, 4, 2, 5, 6, 7],
        [8, 5, 9, 7, 6, 1, 4, 2, 3],
        [4, 2, 6, 8, 5, 3, 7, 9, 1],
        [7, 1, 3, 9, 2, 4, 8, 5, 6],
        [9, 6, 1, 5, 3, 7, 2, 8, 4],
        [2, 8, 7, 4, 1, 9, 6, 3, 5],
        [3, 4, 5, 2, 8, 6, 1, 7, 9]
    ])

    solved_board = input_board.copy()
    solver_sudoku(solved_board)
    np.testing.assert_array_equal(solved_board, expected_solution)


def test_solve_already_solved():
    input_board = np.array([
        [5, 3, 4, 6, 7, 8, 9, 1, 2],
        [6, 7, 2, 1, 9, 5, 3, 4, 8],
        [1, 9, 8, 3, 4, 2, 5, 6, 7],
        [8, 5, 9, 7, 6, 1, 4, 2, 3],
        [4, 2, 6, 8, 5, 3, 7, 9, 1],
        [7, 1, 3, 9, 2, 4, 8, 5, 6],
        [9, 6, 1, 5, 3, 7, 2, 8, 4],
        [2, 8, 7, 4, 1, 9, 6, 3, 5],
        [3, 4, 5, 2, 8, 6, 1, 7, 9]
    ])

    solved_board = input_board.copy()
    solver_sudoku(solved_board)
    np.testing.assert_array_equal(solved_board, input_board)


def test_unsolvable_board():
    input_board = np.array([
        [5, 5, 0, 0, 7, 0, 0, 0, 0],  # Две 5 в первой строке
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ])
    
    assert solver_sudoku(input_board) is False

...                                                                                          [100%]
